In [ ]:
glider_names = [f"ifm{n:02d}" for n in range(1, 16)]
max_depl_num = 99

In [ ]:
print(glider_names)

In [ ]:
import requests
import re
import pandas as pd
from time import sleep

In [ ]:
t, lat, lon = requests.get("https://data.geomar.de/realtime/data/project/ifm14/ifm14_pos.json").json()

In [ ]:
def read_glider_pos_json(glider_name):
    url = f"https://data.geomar.de/realtime/data/project/{glider_name}/{glider_name}_pos.json"
    try:
        glider_pos_data = requests.get(url).json()
    except:
        sleep(3)
        try:
            
            glider_pos_data = requests.get(url).json()
        except:
            glider_pos_data = [None, None, None]
    return {
        "time": glider_pos_data[0],
        "latitude": glider_pos_data[1],
        "longitude": glider_pos_data[2],
    }

In [ ]:
positions = {
    glider_name: read_glider_pos_json(glider_name)
    for glider_name in glider_names
}

In [ ]:
positions = pd.DataFrame.from_dict(positions, orient="index")

In [ ]:
positions["platform"] = positions.index

In [ ]:
positions["time"] = positions["time"].apply(pd.Timestamp)

In [ ]:
positions

In [ ]:
positions["time"].apply(pd.Timestamp)

In [ ]:
!mkdir -p data/
positions.dropna().to_csv("data/gliders.csv")

In [ ]:
!cat data/gliders.csv